In [164]:
import numpy as np
import numpy.linalg
import scipy.linalg

## Linsolve Exact with LU

In [165]:
def lu(A):
    (rows, columns) = A.shape
    L = np.eye(rows)  # Identity matrix for L
    U = A.copy()      # Copy of A for U

    for row in range(rows-1):
        for i in range(row+1, rows):
            scalar = U[i, row] / U[row, row]
            U[i, row:columns] -= U[row, row:columns] * scalar
            L[i, row] = scalar

    return L, U

def forward_backward_substitution(L, U, b):
    (rows, columns) = L.shape

    # solve LZ = b
    z = np.zeros(rows)
    for row in range(rows):
        z[row] = b[row] - np.dot(L[row, :row], z[:row])

    # solve UX = Z
    x = np.zeros(rows)
    for row in reversed(range(rows)):
        x[row] = (z[row] - np.dot(U[row, row+1:], x[row+1:])) / U[row, row]

    return x

# solves Ax = b for x
def lu_linsolve(A, b):
    L, U = lu(A)
    return forward_backward_substitution(L, U, b)


#test linsolve
n = 4
for k in range(1000):
    A = np.random.rand(n,n)
    rhs = np.random.rand(n)
    x = lu_linsolve(A.copy(), rhs)
    assert( np.linalg.norm(rhs - A @ x) < 1e-10)

## Linsolve Least Squares with Cholesky

In [166]:
def cholesky_linsolve(A, b):
    L = np.linalg.cholesky(A.T @ A)
    x = scipy.linalg.solve_triangular(L, A.T @ b, lower=True)
    return scipy.linalg.solve_triangular(L.T, x, lower=False)

x = np.array([5000, 6000])
y = np.array([100, 110])

# create matrix for model function (e.g.): y = p1 + p2 * x
A = np.vstack((np.ones_like(x), x)).T
p1, p2 = cholesky_linsolve(A, y)
print((p1, p2))

(49.99999999999906, 0.010000000000000168)


## Linsolve Least Squares with QR

In [167]:
def back_substitution(U, y):
    n = U.shape[1]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]

    return x

def linsolve_lstsq(A, b):
    num_param = A.shape[1]
    q, r = scipy.linalg.qr(A) # Q*R*p = y
    return back_substitution(r[0:num_param], (q.T @ b)[0:num_param]) # solve for: R*p = Q.T * y

In [168]:
x = np.array([5000, 6000])
y = np.array([100, 110])

# create matrix for model function (e.g.): y = p1 + p2 * x
A = np.vstack((np.ones_like(x), x)).T
p1, p2 = linsolve_lstsq(A, y)
print((p1, p2))

(50.00000000000003, 0.00999999999999999)
